Test Full Pipeline For BMI detection using extra features extracted as well

In [16]:
# conda create -n bmi-predictor python=3.10 -y
# conda activate bmi-predictor

# # For MPS on macOS with ARM (M1/M2/M3)
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

# pip install facenet-pytorch
# pip install pandas scikit-learn matplotlib tqdm jupyter notebook
# pip install opencv-python pillow

### Data Preprocessing

In [2]:
# ===============================================
# STEP 0: Imports and Setup
# ===============================================
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1, MTCNN
import pandas as pd
import numpy as np
from PIL import Image
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm 

# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Core count: {os.cpu_count()}')
torch.set_num_threads(os.cpu_count())  # Use all available CPU cores
DEVICE = torch.device("cpu")  # Force CPU usage for portability
print("Using device:", DEVICE)

Core count: 10
Using device: cpu
Using device: cpu


In [3]:
# Paths
IMAGE_DIR = "data/Images"
existing_image_files = os.listdir(IMAGE_DIR)

# Load CSV
data_df = pd.read_csv("data/data.csv").drop(columns=["Unnamed: 0"])
print(f"Loaded {data_df.shape[0]} rows from CSV")

# Filter out rows with missing image files
data_df = data_df[data_df["name"].isin(existing_image_files)]
print(f"Filtered to {data_df.shape[0]} rows with existing images")

del existing_image_files
data_df.head(5)

Loaded 4206 rows from CSV
Filtered to 3962 rows with existing images


,bmi,gender,is_training,name
0,34.207396,Male,1,img_0.bmp
1,26.453720,Male,1,img_1.bmp
2,34.967561,Female,1,img_2.bmp
3,22.044766,Female,1,img_3.bmp
6,25.845588,Female,1,img_6.bmp


In [20]:
features_df = pd.read_csv('data/Feature_Add.csv')
features_df.head()

,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,left_eye_width,right_eye_width,face_width_to_height,mouth_to_nose_ratio,image,race,pred_gender,age,key,bmi,gender,is_training,name
0,237.002110,210.287898,237.008439,52.009614,120.016666,58.034473,62.008064,46.173586,42.107007,1.127036,2.307586,img_1066_face0.jpg,White,Female,20-29,img_1066,41.191406,Female,1,img_1066.bmp
1,226.035395,200.010000,220.002273,51.000000,104.076895,38.052595,62.072538,43.011626,41.048752,1.130120,2.040723,img_1585_face0.jpg,White,Male,20-29,img_1585,24.658895,Male,1,img_1585.bmp
2,241.101638,209.193690,239.075302,42.011903,87.005747,32.062439,65.000000,42.107007,43.011626,1.152528,2.070978,img_3852_face0.jpg,White,Female,20-29,img_3852,39.151259,Female,0,img_3852.bmp
3,229.490741,198.494332,229.176788,53.037722,80.056230,12.000000,54.083269,42.011903,41.012193,1.156158,1.509421,img_1857_face0.jpg,East Asian,Male,30-39,img_1857,25.845588,Male,1,img_1857.bmp
4,235.552542,201.613492,235.766834,48.041649,77.524190,27.892651,60.299254,34.014703,41.194660,1.168337,1.613687,img_4196_face0.jpg,White,Male,20-29,img_4196,36.243556,Male,0,img_4196.bmp


In [21]:
df = pd.merge(data_df, features_df, on=['gender', 'bmi', 'name', 'is_training'], how='inner')
df.drop(columns=['key', 'image', 'gender'], inplace=True)
print("Combined dataset shape:", df.shape)
df.head()

Combined dataset shape: (3712, 17)


,bmi,is_training,name,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,left_eye_width,right_eye_width,face_width_to_height,mouth_to_nose_ratio,race,pred_gender,age
0,34.207396,1,img_0.bmp,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,62.008064,44.045431,43.046487,1.374692,1.940412,White,Male,30-39
1,26.453720,1,img_1.bmp,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,59.008474,41.012193,43.185646,1.227341,1.832808,White,Male,20-29
2,34.967561,1,img_2.bmp,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,59.033889,42.011903,44.045431,1.240158,2.146238,White,Female,20-29
3,22.044766,1,img_3.bmp,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,59.008474,41.000000,41.012193,1.128941,2.316792,White,Female,20-29
4,25.845588,1,img_6.bmp,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,69.260378,48.259714,47.010637,1.218169,1.937610,White,Female,10-19


In [22]:
#### General information ####
print(f"Entire data shape: {df.shape}")
gender_total = df['pred_gender'].value_counts() / len(df)
gender_total = gender_total.reset_index()
gender_total.rename(columns={'count': 'percentage'}, inplace=True)
print(f"Gender Distribution Over Entire Dataset:\n{gender_total}\n")

# Split into training and testing sets
training_data_df = df[df['is_training'] == 1]
testing_data_df = df[df['is_training'] == 0]

gender_training_data_df = training_data_df['pred_gender'].value_counts() / len(training_data_df)
gender_training_data_df = gender_training_data_df.reset_index()
gender_training_data_df.rename(columns={'count': 'percentage'}, inplace=True)
print(f"Training data shape: {training_data_df.shape}")
print(f"Gender Distribution Over Training Dataset:\n{gender_training_data_df}\n")

gender_testing_data_df = testing_data_df['pred_gender'].value_counts() / len(testing_data_df)
gender_testing_data_df = gender_testing_data_df.reset_index()
gender_testing_data_df.rename(columns={'count': 'percentage'}, inplace=True)
print(f"Testing data shape: {testing_data_df.shape}")
print(f"Gender Distribution Over Testing Dataset:\n{gender_testing_data_df}\n")

del data_df, features_df, gender_total, training_data_df, testing_data_df, gender_training_data_df, gender_testing_data_df

Entire data shape: (3712, 17)
Gender Distribution Over Entire Dataset:
  pred_gender  percentage
0        Male    0.581088
1      Female    0.418912

Training data shape: (3008, 17)
Gender Distribution Over Training Dataset:
  pred_gender  percentage
0        Male    0.590093
1      Female    0.409907

Testing data shape: (704, 17)
Gender Distribution Over Testing Dataset:
  pred_gender  percentage
0        Male    0.542614
1      Female    0.457386



In [23]:
### One Hot Encode ###
race_dummies = pd.get_dummies(df['race'], prefix='race').astype(int)
gender_dummies = pd.get_dummies(df['pred_gender'], prefix='gender').astype(int)
age_dummies = pd.get_dummies(df['age'], prefix='age').astype(int)

# Fix column names immediately after get_dummies
race_dummies.columns = [col.replace(" ", "_") for col in race_dummies.columns]
gender_dummies.columns = [col.replace(" ", "_") for col in gender_dummies.columns]
age_dummies.columns = [col.replace(" ", "_") for col in age_dummies.columns]

# Combine with original numeric features
df_final = pd.concat([
    df.drop(['race', 'pred_gender', 'age'], axis=1),
    race_dummies,
    gender_dummies,
    age_dummies
], axis=1)

del df, race_dummies, gender_dummies, age_dummies
df_final.head()

,bmi,is_training,name,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,...,race_Latino_Hispanic,race_White,gender_Female,gender_Male,age_10-19,age_20-29,age_3-9,age_30-39,age_40-49,age_50-59
0,34.207396,1,img_0.bmp,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,62.008064,...,0,1,0,1,0,0,0,1,0,0
1,26.453720,1,img_1.bmp,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,59.008474,...,0,1,0,1,0,1,0,0,0,0
2,34.967561,1,img_2.bmp,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,59.033889,...,0,1,1,0,0,1,0,0,0,0
3,22.044766,1,img_3.bmp,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,59.008474,...,0,1,1,0,0,1,0,0,0,0
4,25.845588,1,img_6.bmp,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,69.260378,...,0,1,1,0,1,0,0,0,0,0


In [24]:
extra_feature_cols = [col for col in df_final.columns if col not in ['name', 'bmi', 'is_training']]
print(df_final[extra_feature_cols].dtypes[df_final[extra_feature_cols].dtypes == "object"])

Series([], dtype: object)


### Face Detection & Alignment using MTCNN

In [25]:
# ===============================================
# STEP 2: MTCNN for Face Cropping
# ===============================================
# Add margin for help to capture facial shape and surrounding features
# post_process to True to have MTCNN handle resizing and normalization 
mtcnn = MTCNN(image_size=160, margin=20, post_process=True, device=DEVICE)

In [26]:
# ===============================================
# STEP 4: Dataset Class
# ===============================================
class BMIDataset(Dataset):
    def __init__(self, df, image_dir):
        self.df = df.reset_index(drop=True)
        self.image_dir = Path(image_dir)
        
        # Numerical + one-hot encoded feature columns
        self.extra_feature_cols = [col for col in df.columns if col not in ['name', 'bmi', 'is_training']]
        self.scaler = StandardScaler()
        self.df[self.extra_feature_cols] = self.df[self.extra_feature_cols].apply(pd.to_numeric, errors='coerce')
        self.df[self.extra_feature_cols] = self.df[self.extra_feature_cols].fillna(0.0)
        self.df[self.extra_feature_cols] = self.scaler.fit_transform(self.df[self.extra_feature_cols])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = self.image_dir / row['name']

        image = Image.open(image_path).convert('RGB')
        face = mtcnn(image)
        
        if face is None:
            face = torch.zeros((3, 160, 160))  # handle missing face

        # Ensure extra_features is a tensor of float32
        extra_features = torch.tensor(
            np.array(row[self.extra_feature_cols].values, dtype=np.float32)
        )
        bmi = torch.tensor(row['bmi'], dtype=torch.float32)

        return face, extra_features, bmi

In [27]:
# ===============================================
# STEP 5: Model Definition (Multitask)
# ===============================================
class BMIMultitaskModel(nn.Module):
    def __init__(self, extra_feature_dim):
        super().__init__()
        self.backbone = InceptionResnetV1(pretrained='vggface2').eval()
        for param in self.backbone.parameters():
            param.requires_grad = False  # freeze if desired
        
        self.fc = nn.Sequential(
            nn.Linear(512 + extra_feature_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # BMI regression head
        )

    def forward(self, face_img, extra_features):
        face_embed = self.backbone(face_img)
        x = torch.cat([face_embed, extra_features], dim=1)
        return self.fc(x).squeeze(1)

In [28]:
# ===============================================
# STEP 6: Prepare Data Loaders
# ===============================================
train_df = df_final[df_final['is_training'] == 1]
test_df = df_final[df_final['is_training'] == 0]

train_dataset = BMIDataset(train_df, image_dir=IMAGE_DIR)
test_dataset = BMIDataset(test_df, image_dir=IMAGE_DIR)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

In [29]:
# ===============================================
# STEP 7: Training Loop
# ===============================================
model = BMIMultitaskModel(extra_feature_dim=len(train_dataset.extra_feature_cols)).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

def train_epoch(model, dataloader):
    model.train()
    total_loss = 0
    for face, extra, bmi in tqdm(dataloader, desc="Training", leave=False):
        # face, extra, bmi = face.to(DEVICE), extra.to(DEVICE), bmi.to(DEVICE)
        face = face.to(DEVICE, non_blocking=True)
        extra = extra.to(DEVICE, non_blocking=True)
        bmi = bmi.to(DEVICE, non_blocking=True)
        
        optimizer.zero_grad()
        preds = model(face, extra)
        loss = criterion(preds, bmi)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [15]:
# ===============================================
# STEP 8: Evaluation
# ===============================================
def evaluate(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for face, extra, bmi in tqdm(dataloader, desc="Evaluating", leave=False):
            # face, extra, bmi = face.to(DEVICE), extra.to(DEVICE), bmi.to(DEVICE)
            face = face.to(DEVICE, non_blocking=True)
            extra = extra.to(DEVICE, non_blocking=True)
            bmi = bmi.to(DEVICE, non_blocking=True)
            
            preds = model(face, extra)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(bmi.cpu().numpy())
    
    preds = np.concatenate(all_preds)
    labels = np.concatenate(all_labels)
    mae = np.mean(np.abs(preds - labels))
    return mae

# Training loop with early stopping
for epoch in range(10):
    loss = train_epoch(model, train_loader)
    val_mae = evaluate(model, test_loader)
    print(f"Epoch {epoch+1}: Train Loss={loss:.4f}, Val MAE={val_mae:.2f}")

Epoch 1: Train Loss=1105.1727, Val MAE=32.95


Epoch 2: Train Loss=1053.0999, Val MAE=31.62


Epoch 3: Train Loss=919.8919, Val MAE=28.41


Epoch 4: Train Loss=682.8822, Val MAE=22.80


Epoch 5: Train Loss=403.2838, Val MAE=15.36


Epoch 6: Train Loss=200.6451, Val MAE=9.97


Epoch 7: Train Loss=111.6949, Val MAE=7.68


Epoch 8: Train Loss=83.8536, Val MAE=7.02


Epoch 9: Train Loss=73.8280, Val MAE=6.71


Training:  51%|█████     | 48/94 [3:32:56<23:07:01, 1809.17s/it]